In [1]:
import pandas as pd
import numpy as np

In [3]:
db = pd.read_csv("train.csv")
db = db.reindex(np.random.permutation(db.index))
comment = db['comment_text'].as_matrix()
label = db[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']].as_matrix()
# print (comments)
comments = []
labels = []

for ix in range(comment.shape[0]):
    if len(comment[ix])<=700:
        comments.append(comment[ix])
        labels.append(label[ix])

In [4]:
labels = np.asarray(labels)

In [5]:
print(len(comments))

82464


In [6]:
from stop_words import get_stop_words
stop_words = get_stop_words('english')
stop_words.append('')

for x in range(ord('b'), ord('z')+1):
    stop_words.append(chr(x))

In [7]:
print (stop_words)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

In [8]:
import string
punctuation_edit = string.punctuation.replace('\'','') +"0123456789"
print ((punctuation_edit))
print (comments[1])
outtab = "                                         "
trantab = str.maketrans(punctuation_edit, outtab)

!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789
He'll get his at the arbitration where his edits won't stand. Insults can go one way around here from the violoator to the just. He'll lose.


In [9]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [10]:
lemmatiser = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/paavini/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
for i in range(len(comments)):
    comments[i] = comments[i].lower().translate(trantab)
    l = []
    for word in comments[i].split():
        l.append(stemmer.stem(lemmatiser.lemmatize(word,pos="v")))
    comments[i] = " ".join(l)

KeyboardInterrupt: 

In [ ]:
count_vector = CountVectorizer(stop_words=stop_words)
tf = count_vector.fit_transform(comments).toarray()

In [ ]:
print(tf.shape)

In [ ]:
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion)
    print(ratio)
    X_train = matrix[ratio:,:]
    X_test =  matrix[:ratio,:]
    Y_train = target[ratio:,:]
    Y_test =  target[:ratio,:]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = shuffle(tf, labels, 3)

print(Y_train.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [ ]:
model = Sequential()
model.add(Dense(2, activation='relu', input_dim = X_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.myneuralfull.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(X_train, Y_train, epochs=10, batch_size=32)

In [ ]:
model.load_weights('saved_models/weights.best.myneuralfull.hdf5')

In [ ]:
predict = model.predict(X_test)

In [ ]:
predict = np.round(predict)

In [ ]:
print(predict.shape)

In [ ]:
from sklearn.metrics import hamming_loss
loss = hamming_loss(Y_test,predict)
print(loss*100)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test,predict)
print(accuracy*100)

In [ ]:
from sklearn.metrics import log_loss
print(log_loss(Y_test,predict))